In [1]:
#------Importing Libraries------#
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVR
from sklearn.ensemble import GradientBoostingRegressor, RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report, mean_squared_error, accuracy_score, r2_score
import seaborn as sns

In [2]:
#---------Data loading and preprocessing---------#

# Load the data
data = pd.read_csv('proj-data.csv')

# Data preprocessing
# Remove trailing colons from column names
data.columns = data.columns.str.rstrip(':').str.strip()

# Replace non-numeric values with NaN
data.replace('?', np.nan, inplace=True)

# Handle binary columns
binary_columns = ['TSH measured', 'T3 measured', 'TT4 measured', 'T4U measured', 'FTI measured', 'TBG measured']
for col in binary_columns:
    data[col] = data[col].map({'f': 0, 't': 1})

# Convert appropriate columns to numeric
numeric_columns = ['age', 'TSH', 'T3', 'TT4', 'T4U', 'FTI', 'TBG']
for col in numeric_columns:
    data[col] = pd.to_numeric(data[col], errors='coerce')

# Check for negative values before imputation
print("Negative values before imputation in age:", (data['age'] < 0).sum())

# Handle missing values for numeric columns
imputer = SimpleImputer(strategy='mean')
data[numeric_columns] = imputer.fit_transform(data[numeric_columns])

# Check for negative values after imputation
print("Negative values after imputation in age:", (data['age'] < 0).sum())

# Encode categorical variables
label_encoders = {}
categorical_columns = ['sex', 'on thyroxine', 'query on thyroxine', 'on antithyroid medication', 'sick', 'pregnant',
                       'thyroid surgery', 'I131 treatment', 'query hypothyroid', 'query hyperthyroid', 'lithium', 
                       'goitre', 'tumor', 'hypopituitary', 'psych', 'referral source', 'diagnoses']
for column in categorical_columns:
    label_encoders[column] = LabelEncoder()
    data[column] = label_encoders[column].fit_transform(data[column].astype(str))

# Preserve the record identification column
record_identification = data['[record identification]'].values

# Drop the record identification column from the main DataFrame
data.drop(columns=['[record identification]'], inplace=True)

Negative values before imputation in age: 0
Negative values after imputation in age: 0


In [3]:
#--------Debugging--------#

# Ensure all data is numeric
#print(data.dtypes)

# Check for non-numeric data in the dataset
for col in data.columns:
    if data[col].dtype == 'object':
        print(f"Column {col} contains non-numeric values.")
        print(data[col].unique())

In [4]:
#----------Label Encoding and mapping----------#

# Define the mapping for new classes
diagnosis_mapping = {
    '-': 'healthy',
    'A': 'hyperthyroid conditions', 'B': 'hyperthyroid conditions', 'C': 'hyperthyroid conditions', 'D': 'hyperthyroid conditions',
    'E': 'hypothyroid conditions', 'F': 'hypothyroid conditions', 'G': 'hypothyroid conditions', 'H': 'hypothyroid conditions',
    'I': 'binding protein', 'J': 'binding protein',
    'K': 'general health',
    'L': 'replacement therapy', 'M': 'replacement therapy', 'N': 'replacement therapy',
    'R': 'discordant results'
}

# Add remaining classes to 'other'
original_diagnoses_labels = label_encoders['diagnoses'].classes_
for label in original_diagnoses_labels:
    if label not in diagnosis_mapping:
        diagnosis_mapping[label] = 'other'

# Map the diagnoses to new classes
data['diagnoses'] = data['diagnoses'].map(lambda x: diagnosis_mapping[label_encoders['diagnoses'].inverse_transform([x])[0]])

# Encode the new class labels
label_encoders['diagnoses'] = LabelEncoder()
data['diagnoses'] = label_encoders['diagnoses'].fit_transform(data['diagnoses'])

In [5]:
#--------Diagnose prediction model--------#

X_diagnosis = data.drop(['diagnoses', 'referral source'], axis=1)
y_diagnosis = data['diagnoses']

# Split data into training and testing sets for diagnosis prediction
X_train_diag, X_test_diag, y_train_diag, y_test_diag = train_test_split(X_diagnosis, y_diagnosis, test_size=0.2, random_state=42)

# Model training for diagnosis prediction
dt_classifier = DecisionTreeClassifier()
dt_classifier.fit(X_train_diag, y_train_diag)

# Model evaluation for diagnosis prediction
y_pred_diag = dt_classifier.predict(X_test_diag)

# Retrieve original class labels for diagnoses
new_diagnoses_labels = label_encoders['diagnoses'].classes_

# Print class labels and their corresponding diagnoses
for class_label, diagnosis in enumerate(new_diagnoses_labels):
    print(f"Class {class_label}: {diagnosis}")

print(classification_report(y_test_diag, y_pred_diag, target_names=new_diagnoses_labels))

# Create a DataFrame to display the results with record identification
predictions_df = pd.DataFrame({'Actual diagnose': y_test_diag, 'Predicted diagnose': y_pred_diag})
print(predictions_df)
print('MSE:', mean_squared_error(y_test_diag, y_pred_diag))

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_diag, y_pred_diag))
print('RMSE:', rmse)

# Calculate accuracy
accuracy = accuracy_score(y_test_diag, y_pred_diag)
print('Accuracy score:', accuracy)

# Feature importance for diagnosis prediction
importances_diag = dt_classifier.feature_importances_
feature_importance_dt_diag = pd.DataFrame({'Feature': X_diagnosis.columns, 'Importance': importances_diag})
print(feature_importance_dt_diag.sort_values(by='Importance', ascending=False))

Class 0: binding protein
Class 1: discordant results
Class 2: general health
Class 3: healthy
Class 4: hyperthyroid conditions
Class 5: hypothyroid conditions
Class 6: other
Class 7: replacement therapy
                         precision    recall  f1-score   support

        binding protein       0.74      0.71      0.72        55
     discordant results       0.81      0.71      0.76        31
         general health       0.82      0.83      0.83        72
                healthy       0.97      0.97      0.97      1114
hyperthyroid conditions       0.78      0.72      0.75        25
 hypothyroid conditions       0.92      0.99      0.95        81
                  other       0.74      0.79      0.76        33
    replacement therapy       0.96      0.93      0.95        57

               accuracy                           0.94      1468
              macro avg       0.84      0.83      0.84      1468
           weighted avg       0.94      0.94      0.94      1468

      Actual d

In [6]:
#---------Age prediction model---------#

X_age = data.drop(['age', 'referral source'], axis=1)
y_age = data['age']

#data_types = X_age.dtypes
#print('Data types: ', data_types)

# DEBUGGING
# Identify continuous and discrete features
#continuous_features = [col for col, dtype in data_types.items() if dtype == 'float64']
#discrete_features = [col for col, dtype in data_types.items() if dtype == 'int64']

#print("Continuous Features:", continuous_features)
#print("Discrete Features:", discrete_features)

# Removing outliers (example using IQR method)
Q1 = y_age.quantile(0.25)
Q3 = y_age.quantile(0.75)
IQR = Q3 - Q1
outlier_mask = (y_age >= (Q1 - 1.5 * IQR)) & (y_age <= (Q3 + 1.5 * IQR))

X_age_no_outliers = X_age[outlier_mask]
y_age_no_outliers = y_age[outlier_mask]
record_identification_no_outliers = record_identification[outlier_mask]

#X_age_discrete = X_age_no_outliers.select_dtypes(exclude=['float64'])

# Split data into training and testing sets for age prediction
X_train_age, X_test_age, y_train_age, y_test_age, record_train, record_test = train_test_split(X_age_no_outliers, y_age_no_outliers, record_identification_no_outliers, test_size=0.2, random_state=42)

# Apply feature scaling
scaler = StandardScaler()
X_train_age = scaler.fit_transform(X_train_age)
X_test_age = scaler.transform(X_test_age)

# Model training for age prediction
gbr_regressor = GradientBoostingRegressor()
gbr_regressor.fit(X_train_age, y_train_age)
print("(GBR)Negative X_age:", (gbr_regressor.predict(X_test_age) < 0).sum())

rfr_regressor = RandomForestRegressor()
rfr_regressor.fit(X_train_age, y_train_age)
print("(RFR)Negative X_age:", (rfr_regressor.predict(X_test_age) < 0).sum())

svr_regressor = SVR()
svr_regressor.fit(X_train_age, y_train_age)
print("(SVR)Negative X_age:", (svr_regressor.predict(X_test_age) < 0).sum())

# Age prediction evaluation
y_pred_age_gbr = gbr_regressor.predict(X_test_age)
y_pred_age_rfr = rfr_regressor.predict(X_test_age)
y_pred_age_svr = svr_regressor.predict(X_test_age)

#---------------------------------Gradient Boosting Regressor---------------------------------#
# Create a DataFrame to display the results with record identification
predictions_df = pd.DataFrame({'Record Identification': record_test, 'Actual Age': y_test_age, 'Predicted Age': y_pred_age_gbr})
print(predictions_df)
print('MSE:', mean_squared_error(y_test_age, y_pred_age_gbr))

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_age, y_pred_age_gbr))
print('RMSE:', rmse)

# Calculate accuracy
accuracy_age = gbr_regressor.score(X_test_age, y_test_age)
print('Accuracy score:', accuracy_age)

# Calculate R2 score
r2 = r2_score(y_test_age, y_pred_age_gbr)
print('R2 score:', r2)

# Feature importance for age prediction
importances_age = gbr_regressor.feature_importances_
feature_importance_rf_age = pd.DataFrame({'Feature': X_age_no_outliers.columns, 'Importance': importances_age})
print(feature_importance_rf_age.sort_values(by='Importance', ascending=False))

# Plot predicted vs. actual ages
plt.scatter(y_test_age, y_pred_age_gbr, alpha=0.3)
plt.xlabel('Actual Age')
plt.ylabel('Predicted Age')
plt.title('Predicted vs. Actual Age')
plt.show()

#---------------------------------Random Forest Regressor---------------------------------#
# Create a DataFrame to display the results with record identification
predictions_df2 = pd.DataFrame({'Record Identification': record_test, 'Actual Age': y_test_age, 'Predicted Age': y_pred_age_rfr})
print(predictions_df2)
print('MSE:', mean_squared_error(y_test_age, y_pred_age_rfr))

# Calculate RMSE
rmse2 = np.sqrt(mean_squared_error(y_test_age, y_pred_age_rfr))
print('RMSE:', rmse2)

# Calculate accuracy
accuracy_age2 = gbr_regressor.score(X_test_age, y_test_age)
print('Accuracy score:', accuracy_age2)

# Calculate R2 score
r2 = r2_score(y_test_age, y_pred_age_rfr)
print('R2 score:', r2)

# Feature importance for age prediction
importances_age2 = rfr_regressor.feature_importances_
feature_importance_rfr_age = pd.DataFrame({'Feature': X_age_no_outliers.columns, 'Importance': importances_age2})
print(feature_importance_rfr_age.sort_values(by='Importance', ascending=False))

# Plot predicted vs. actual ages
plt.scatter(y_test_age, y_pred_age_rfr, alpha=0.3)
plt.xlabel('Actual Age')
plt.ylabel('Predicted Age')
plt.title('Predicted vs. Actual Age')
plt.show()

#---------------------------------SVR---------------------------------#
# Create a DataFrame to display the results with record identification
predictions_df3 = pd.DataFrame({'Record Identification': record_test, 'Actual Age': y_test_age, 'Predicted Age': y_pred_age_svr})
print(predictions_df2)
print('MSE:', mean_squared_error(y_test_age, y_pred_age_svr))

# Calculate RMSE
rmse3 = np.sqrt(mean_squared_error(y_test_age, y_pred_age_svr))
print('RMSE:', rmse3)

# Calculate accuracy
accuracy_age3 = gbr_regressor.score(X_test_age, y_test_age)
print('Accuracy score:', accuracy_age2)

# Calculate R2 score
r2 = r2_score(y_test_age, y_pred_age_svr)
print('R2 score:', r2)

# Feature importance for age prediction
importances_age3 = svr_regressor.feature_importances_
feature_importance_svr_age = pd.DataFrame({'Feature': X_age_no_outliers.columns, 'Importance': importances_age3})
print(feature_importance_svr_age.sort_values(by='Importance', ascending=False))

# Plot predicted vs. actual ages
plt.scatter(y_test_age, y_pred_age_svr, alpha=0.3)
plt.xlabel('Actual Age')
plt.ylabel('Predicted Age')
plt.title('Predicted vs. Actual Age')
plt.show()

(GBR)Negative X_age: 0
(RFR)Negative X_age: 0
(SVR)Negative X_age: 0


NameError: name 'y_pred_age' is not defined

In [ ]:
#---------Sex prediction model---------#

# Gender prediction model
X_gender = data.drop(['sex', 'referral source'], axis=1)
y_gender = data['sex']

# Split data into training and testing
X_train_gender, X_test_gender, y_train_gender, y_test_gender, record_train, record_test = train_test_split(X_gender, y_gender, record_identification, test_size=0.2, random_state=42)

# Model training
rf_classifier = RandomForestClassifier()
rf_classifier.fit(X_train_gender, y_train_gender)

y_pred_gender = rf_classifier.predict(X_test_gender)

predictions2_df = pd.DataFrame({'Record Identification': record_test, 'Actual gender': y_test_gender, 'Predicted sex': y_pred_gender})
print(predictions2_df)

# Feature importance for sex prediction
importances_gender = rf_classifier.feature_importances_
feature_importance_rf_gender = pd.DataFrame({'Feature': X_gender.columns, 'Importance': importances_gender})
print(feature_importance_rf_gender.sort_values(by='Importance', ascending=False))

print(classification_report(y_test_gender, y_pred_gender, target_names=label_encoders['sex'].classes_))

# Create a DataFrame to display the results with record identification
predictions_df = pd.DataFrame({'Record Identification': record_test, 'Actual gender': y_test_gender, 'Predicted gender': y_pred_gender})
print(predictions_df)
print('MSE:', mean_squared_error(y_test_gender, y_pred_gender))

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test_gender, y_pred_gender))
print('RMSE:', rmse)

# Calculate accuracy
accuracy_gender = accuracy_score(y_test_gender, y_pred_gender)
print('Accuracy score:', accuracy_gender)
